# Mount Google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/NLPwChoo_project/proj3_mt')

# Import libraries

In [ ]:
!pip install wandb numpy transformers torchinfo torch lib datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7

In [ ]:
# Log in to your W&B account
import wandb
wandb.login(key='8985569cfbb8c8c724e2b5245d63589a746bf6cd')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="groom_proj3_dataset_en_ko")

wandb: Currently logged in as: seonhye (team-nlpk). Use `wandb login --relogin` to force relogin


In [ ]:
import random
import numpy as np
import torch 
import evaluate
import pandas as pd

from typing import Dict, List
import csv


from transformers import (
    EncoderDecoderModel, 
    GPT2Tokenizer as BaseGPT2Tokenizer,
    PreTrainedTokenizer, 
    PreTrainedTokenizerFast,
    ElectraTokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Trainer,
    AutoConfig,
    AutoTokenizer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,

    Trainer,
    TrainingArguments
)

import torch

from transformers.models.encoder_decoder.modeling_encoder_decoder import EncoderDecoderModel

from torchinfo import summary

In [ ]:
# free up GPU memory
torch.cuda.empty_cache()

In [ ]:
def seed_everthing(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everthing(42)

In [ ]:
class PairedDataset:
    def __init__(self, 
        src_tokenizer: PreTrainedTokenizer, tgt_tokenizer: PreTrainedTokenizer,
        file_path: str
    ):
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = tgt_tokenizer
        with open(file_path, 'r') as fd:
            self.data = [row[1:] for row in csv.reader(fd)]

    def __getitem__(self, index: int) -> Dict[str, torch.Tensor]:
        src, trg = self.data[index]
        trg += self.trg_tokenizer.eos_token
        embeddings = self.src_tokenizer(src, return_attention_mask=False, return_token_type_ids=False)
        embeddings['labels'] = self.trg_tokenizer(trg, return_attention_mask=False)['input_ids']

        return embeddings

    def __len__(self):
        return len(self.data)

In [ ]:
# class GPT2Tokenizer(BaseGPT2Tokenizer):
#     def build_inputs_with_special_tokens(self, token_ids: List[int], _) -> List[int]:
#         return token_ids + [self.eos_token_id]

In [ ]:
src_tokenizer = AutoTokenizer.from_pretrained('google/electra-small-discriminator')
trg_tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')

dataset = PairedDataset(src_tokenizer, trg_tokenizer, './input/문어체_train.csv')
eval_dataset = PairedDataset(src_tokenizer, trg_tokenizer, './input/문어체_val.csv')

In [ ]:
print(src_tokenizer)
print(trg_tokenizer)

ElectraTokenizerFast(name_or_path='google/electra-small-discriminator', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'})


In [ ]:
print(trg_tokenizer.decode(dataset[1]['labels']))
print(src_tokenizer.decode(dataset[1]['input_ids']))

참가 학생 모두 그날 배운 것을 그날 완전히 소화하기 위해서는 무엇보다 눈 맞춤 수업이 중요하다.</s>
[CLS] eye - tailored classes are most important for all participants to fully digest what they learned that day. [SEP]


In [ ]:
print(src_tokenizer.tokenize("eye - tailored classes are most important for all participants to fully digest what they learned that day."))
print(trg_tokenizer.tokenize("참가 학생 모두 그날 배운 것을 그날 완전히 소화하기 위해서는 무엇보다 눈 맞춤 수업이 중요하다."))

['eye', '-', 'tailored', 'classes', 'are', 'most', 'important', 'for', 'all', 'participants', 'to', 'fully', 'digest', 'what', 'they', 'learned', 'that', 'day', '.']
['▁참가', '▁학생', '▁모두', '▁그날', '▁배운', '▁것을', '▁그날', '▁완전히', '▁소화', '하기', '▁위해서는', '▁무엇보다', '▁눈', '▁맞춤', '▁수', '업이', '▁중요', '하다.']


In [ ]:
print(dataset.__getitem__(1))
print(eval_dataset.__getitem__(1))

{'input_ids': [101, 3239, 1011, 21727, 4280, 2024, 2087, 2590, 2005, 2035, 6818, 2000, 3929, 17886, 2054, 2027, 4342, 2008, 2154, 1012, 102], 'labels': [11181, 10364, 9432, 23398, 23696, 9290, 23398, 11130, 14931, 9154, 11357, 23879, 9847, 21657, 9025, 12208, 9861, 19572, 1]}

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    'google/electra-small-discriminator',
    'skt/kogpt2-base-v2',
    pad_token_id=trg_tokenizer.bos_token_id
)

model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.5.crossattention.q_attn.weight', 'transformer.h.6.crossattention.c_attn.weight', 'transformer.h.5.crossattention.bias', 'transformer.h.4.crossattention.c_attn.weight', 'transformer.h.9.ln_cross_attn.weight', 'transformer.h.5.crossattention.c_proj.weight', 'transformer.h.2.crossattention.c_attn.weight', 'transformer.h.8.ln_cross_attn.weight', 'transformer.h.3.crossattention.bias', 'transformer.h.6.crossattention.c_proj.bias', 'transformer.h.1.crossattention.masked_bias', 'transformer.h.6.crossattention.q_attn.weight', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.6.crossattention.masked_bias', 'transformer.h.2.crossattention.c_proj.bias', 'transformer.h.4.crossattention.q_attn.weight', 'transformer.h.7.crossattention.c_proj.weight', 'transformer.h.8.crossattention.c_attn.weight', 'transformer.h.7.ln_cross_attn.weight', 'transfor

In [ ]:
collator = DataCollatorForSeq2Seq(src_tokenizer, model)

arguments = Seq2SeqTrainingArguments(
    output_dir='dump_plus_eos',
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    gradient_accumulation_steps=4,
    save_total_limit=5,
    dataloader_num_workers=2,
    fp16=True,
    load_best_model_at_end=True,
    report_to = 'wandb'
)

trainer = Trainer(
    model,
    arguments,
    data_collator=collator,
    train_dataset=dataset,
    eval_dataset=eval_dataset
    # compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

PyTorch: setting up devices
Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 350001
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 54690
  Number of trainable parameters = 167211264
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a ElectraTokenizerFast tokenizer. Please note that wi

Epoch,Training Loss,Validation Loss
1,2.352000,2.181499
2,1.932400,1.854082


***** Running Evaluation *****
  Num examples = 50001
  Batch size = 16
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to dump_plus_eos/checkpoint-5469
Configuration saved in dump_plus_eos/checkpoint-5469/config.json
Configuration saved in dump_plus_eos/checkpoint-5469/generation_config.json
Model weights saved in dump_plus_eos/checkpoint-5469/pytorch_model.bin
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` metho

In [ ]:
model.save_pretrained("output_dir='dump_plus_eos/best_model")

# demo

In [ ]:
demo_sentence='Researchers learn in class is different from what researchers experience in the field.'
demo_input=src_tokenizer(demo_sentence, return_tensors="pt")
demo_ids=demo_input.input_ids.to("cuda")

outputs=model.generate(demo_ids, max_new_tokens=512)
output_str=trg_tokenizer.decode(outputs[0],skip_special_tokens=True)

print(output_str)

Generate config GenerationConfig {
  "decoder_start_token_id": 1,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



연구자들은 현장에서 배운 연구자들이 현장에서 배운 것보다 더 많은 것을 배운다.


In [ ]:
demo_sentence='according to the overseas construction general information service of the overseas construction association, the cumulative orders for overseas construction increased 6 % ( usd 1. 1 billion ) this year compared to the same period of the previous year and it is also a positive factor.'
demo_input=src_tokenizer(demo_sentence, return_tensors="pt")
demo_ids=demo_input.input_ids.to("cuda")

outputs=model.generate(demo_ids, max_new_tokens=512)
output_str=trg_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(output_str)
print(trg_tokenizer.decode(outputs[0]))

Generate config GenerationConfig {
  "decoder_start_token_id": 1,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



국내 건설협회 물류정보시스템에 따르면 올해 해외건설 수주액은 전년 대비 4.6% 증가한 3조 3,300만 달러(한화 약 4,300만 원)로 집계됐다.
</s> 국내 건설협회 물류정보시스템에 따르면 올해 해외건설 수주액은 전년 대비 4.6% 증가한 3조 3,300만 달러(한화 약 4,300만 원)로 집계됐다.</s>


In [ ]:
demo_sentence='asked if he had a " will to denuclearize, " chairman kim jong - un replied, " if it had not been for it, would i have come this far? " but the denuclearization he thinks was different from what we had thought.'
demo_input=src_tokenizer(demo_sentence, return_tensors="pt")
demo_ids=demo_input.input_ids.to("cuda")

outputs=model.generate(demo_ids, max_new_tokens=512)
output_str=trg_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(output_str)
print(trg_tokenizer.decode(outputs[0]))

Generate config GenerationConfig {
  "decoder_start_token_id": 1,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



김 위원장은 “(비핵화를) 안 하겠다고 하면 (비핵화) 의지가 있느냐”는 질문에 “그렇지 않다”고 답했지만 비핵화 의지가 있느냐는 질문에는 “그렇지 않다”고 답했다.
</s> 김 위원장은 “(비핵화를) 안 하겠다고 하면 (비핵화) 의지가 있느냐”는 질문에 “그렇지 않다”고 답했지만 비핵화 의지가 있느냐는 질문에는 “그렇지 않다”고 답했다.</s>


# Generating the dataset, which contains Korean informal and formal pairs

In [ ]:
test = pd.read_csv('./input/spoken2eng_final.csv')
test

,Unnamed: 0,prediction,Korean
0,0,They say that it is good to wash apples well a...,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.
1,1,Until when should I go there?,내가 언제까지 거기 가면 됩니까?
2,2,Should we fill it out and deliver it?,우리가 작성해서 전달해 드려야 하나요?
3,3,"I'm happy now, the world wants me.","난 지금 행복해요, 세상이 나를 원하고 있죠."
4,4,He must have been at the flower shop.,그는 틀림없이 꽃 가게에 들렸을 겁니다.
...,...,...,...
99995,99995,Then it would be a good idea to change the con...,그럼 컨셉을 약간 바꾸는 것도 좋은 방법이겠네요.
99996,99996,The password is written in the room.,객실 안에 비밀번호가 작성되어 있습니다.
99997,99997,Can I find it right away when I go to gate 4?,4번 게이트로 갔을 때 바로 찾을 수 있나요?
99998,99998,"3 of them are missing, so is it okay to get th...",지금 3개나 빠진 상태인데 한꺼번에 이렇게 많이 빠져도 상관없는 건가요?


In [ ]:
test.rename(columns = {'prediction': 'spoken_en', 'Korean': 'spoken_ko'}, inplace=True)
test

In [ ]:
p1=[]
for i in range(int(len(test)/10)):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
#   print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p1.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p1))

In [ ]:
p2=[]
for i in range(int(len(test)/10),int(2*(len(test)/10))):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p2.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p2))

In [ ]:
p3=[]
for i in range(int(2*(len(test)/10)),int(3*(len(test)/10))):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p3.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p3))

In [ ]:
p4=[]
for i in range(int(3*(len(test)/10)),int(4*(len(test)/10))):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p4.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p4))

In [ ]:
p5=[]
for i in range(int(4*(len(test)/10)),int(5*(len(test)/10))):
  text = test['en_spoken'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p5.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p5))

In [ ]:
p6=[]
for i in range(int(5*(len(test)/10)),int(6*(len(test)/10))):
  text = test['en_spoken'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p6.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p6))

In [ ]:
p7=[]
for i in range(int(6*(len(test)/10)),int(7*(len(test)/10))):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p7.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p7))

In [ ]:
p8=[]
for i in range(int(7*(len(test)/10)),int(8*(len(test)/10))):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p8.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p8))

In [ ]:
p9=[]
for i in range(int(8*(len(test)/10)),int(9*(len(test)/10))):
  text = test['en_spoken'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p9.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p9))

In [ ]:
p10=[]
for i in range(int(9*(len(test)/10)),int(10*(len(test)/10))):
  text = test['spoken_en'][i]
  embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
  #print(text, embeddings)
  embeddings = {k: v.cuda() for k, v in embeddings.items()}
  output = model.generate(**embeddings, max_new_tokens=512)[0, 1:-1]
  p10.append(trg_tokenizer.decode(output.cpu()))
  print(trg_tokenizer.decode(output.cpu()),len(p10))

In [ ]:
test_predicted = pd.DataFrame({'spoken_ko' : test['spoken_ko'], 'spoken_en' : test['spoken_en'], 'written_en' : test['en'], 'written_ko' : p1+p2+p3+p4+p5+p6+p7+p8+p9+p10})
df_s2w_ko = test_predicted[['spoken_ko', 'written_ko']]

In [ ]:
test_predicted.to_csv('./output/test_predicted.csv', index=True, encoding = "utf-8-sig")
df_s2w_ko.to_csv('./output/df_s2w_ko.csv', index=True, encoding = "utf-8-sig")

In [ ]:
df = pd.read_csv('./output/df_s2w_ko.csv')
df

# Reference

- https://huggingface.co/skt/kogpt2-base-v2
- https://huggingface.co/docs/transformers/v4.19.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizerFast
- https://github.com/huggingface/transformers/issues/17809


- https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb
- https://github.com/google-research/bleurt/tree/master